In [3]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-25 16:10:42--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2021-04-25 16:10:43 (10.8 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [30]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True , dateFormat="yyyy-MM-dd")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [53]:
#checking the data
#check schema and data typs
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [54]:
#check counts and summary - don't run it for a large data set
df.describe().show()

+-------+-----------+--------------------+--------------+--------------------+--------------------+--------------------+----------------+------------------+------------------+------------------+-------+-----------------+---------------+--------------------+-----------+
|summary|marketplace|         customer_id|     review_id|          product_id|      product_parent|       product_title|product_category|       star_rating|     helpful_votes|       total_votes|   vine|verified_purchase|review_headline|         review_body|review_date|
+-------+-----------+--------------------+--------------+--------------------+--------------------+--------------------+----------------+------------------+------------------+------------------+-------+-----------------+---------------+--------------------+-----------+
|  count|    2402458|             2402458|       2402458|             2402458|             2402458|             2402458|         2402434|           2402431|           2402431|           2402

In [62]:
import pyspark.sql.functions as F  

In [66]:
df.agg(F.countDistinct("customer_id"),F.countDistinct("product_id"),F.countDistinct("review_id"),F.count("review_id")).collect()

[Row(count(customer_id)=1363986, count(product_id)=305512, count(review_id)=2402458, count(review_id)=2402458)]

In [71]:
df.agg(F.count("review_id"),F.countDistinct("review_id"),F.sum("star_rating"),F.sum("helpful_votes"),F.sum("total_votes")).collect()

[Row(count(review_id)=2402458, count(review_id)=2402458, sum(star_rating)=10359832, sum(helpful_votes)=3712495, sum(total_votes)=4733255)]

### Create DataFrames to match tables

In [31]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [63]:
# Create the customers_table DataFrame
#Customer table has columns customer_id and   customer_count. So use a group by on the dataframe to get customer dataframe
#To create the customers_table, Use the groupby() function on the customer_id column 
#Count all the customer ids using the agg() function by chaining it to the groupby() function. 
#After you use this function, a new column will be created, count(customer_id).
#Rename the count(customer_id) column using the withColumnRenamed() function so it matches the schema for the customers_table in pgAdmin.
customers_df = df.groupby("customer_id").count().withColumnRenamed("count","customer_count")


In [59]:

#check customer_df
customers_df.agg(F.sum("customer_count"),F.count("customer_id")).collect()
 #customer count and no of customer matches with the the input data frame df

[Row(sum(customer_count)=2402458, count(customer_id)=1363986)]

In [51]:
# Create the products_table DataFrame and drop duplicates. 
#Product table has columns product_id, product_title 
products_df = df.select(["product_id","product_title"]).drop_duplicates()
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0088PCJ1C|Bob's Pickle Pops...|
|B005P0U5BO|Envirokidz Organi...|
|B00K2RY8GI|Coffee Variety Sa...|
|B008233EI2|Once Again Organi...|
|B0029K0U90|Mrs. Wages ALL NA...|
|B003H4AMEU|Bhuja Crunchy Sea...|
|B004AJIG76|Krusteaz Belgian ...|
|B002LME6OQ|Prince of Peace O...|
|B00NP7U2U8|Yoki - Cheese Bre...|
|B00ISQD2YO|Nescafe with Coff...|
|B003X35RR2|Bob's Red Mill Gl...|
|B00WVSNJ8Y|Organic Valley, O...|
|B0002UM13C|Sour Cherry Prese...|
|B0042M7PS2|Rodelle Gourmet B...|
|B002DVPTVO|Coffee Go Candy -...|
|B008EL7N2U|Melitta Coffee Gr...|
|B00CCS8Y5Q|Trader Joe's Coco...|
|B006ZOXLF6|Flossugar (1/2 ga...|
|B004RJ0C1W|Camp Chicory & Co...|
|B005JJZ430|KONA BLEND COFFEE...|
+----------+--------------------+
only showing top 20 rows



In [87]:
#check products_df
products_df.agg(F.count("product_id"),F.countDistinct("product_id")).collect()
#looks like the distinct product id in input data frame has slightly higher number due to product title might be different for same product id
# due to duplicate we won't be able to insert in the table as product id is primary key and it will not load.

[Row(count(product_id)=305538, count(product_id)=305512)]

In [82]:
#check duplicate rows in the porducts datafrae
prod_dup_df =products_df.groupby("product_id" ).count().filter("count >1") 
prod_dup_df.orderBy("product_id").show()


+----------+-----+
|product_id|count|
+----------+-----+
|B004BUU488|   27|
+----------+-----+



In [85]:
#now check the title for these product id where multiple rows exists
prod_dup_title_df = products_df.join(prod_dup_df,on="product_id",how="inner") 
prod_dup_title_df.show(truncate=False)

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [89]:
#based on the above data , it seems like there was some problem in reading these rows. so for now we will delete these records.
products_df=products_df.where("product_id != 'B004BUU488'")

products_df.agg(F.count("product_id"),F.countDistinct("product_id")).collect()


[Row(count(product_id)=305511, count(product_id)=305511)]

In [90]:
# Create the review_id_table DataFrame. 
#reivew table has following columns review_id, customer_id,product_id,product_parent,review_date is a date.
#as review date is a date but in the dataframe is a string , we should convert it to date.
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")]).where("product_id != 'B004BUU488'")
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R26MV8D0KG6QI6|   42521656|B000SAQCWC|     159713740| 2015-08-31|
|R1OF8GP57AQ1A0|   12049833|B00509LVIQ|     138680402| 2015-08-31|
|R3VDC1QB6MC4ZZ|     107642|B00KHXESLC|     252021703| 2015-08-31|
|R12FA3DCF8F9ER|    6042304|B000F8JIIC|     752728342| 2015-08-31|
| RTWHVNV6X4CNJ|   18123821|B004ZWR9RQ|     552138758| 2015-08-31|
| RIG9AWFOGRDVO|   23649464|B00AL6QBZ6|     681475449| 2015-08-31|
|R1S1XSG4ZCHDGS|   32778285|B00BCNSTRA|     578681693| 2015-08-31|
| RB15NBVY5ELVW|   46612941|B0089MM2BG|     350968436| 2015-08-31|
| R56358YM1ZJ7I|   31525399|B00Y1C9770|     729982780| 2015-08-31|
|R1ODXB3C9UP3NL|   19624355|B00J074W94|       2499702| 2015-08-31|
|R155ZMVLD5C9BP|   31910375|B0001VKKOO|     189938205| 2015-08-31|
|R2IXW43PUNYM0G|   12816533|B007TGH4CK|     398128262| 2015-08

In [91]:
review_id_df.agg(F.count("review_id"),F.countDistinct("review_id"),F.min("review_date"),F.max("review_date")).collect()

[Row(count(review_id)=2402431, count(review_id)=2402431, min(review_date)=datetime.date(1999, 9, 5), max(review_date)=datetime.date(2015, 8, 31))]

In [92]:
# Create the vine_table. DataFrame
#Vine table has following columns review_id ,  star_rating ,  helpful_votes ,  total_votes ,  vine ,  verified_purchase 
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"]).where("product_id != 'B004BUU488'")
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26MV8D0KG6QI6|          5|            0|          0|   N|                Y|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|                Y|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|                N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|                Y|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|                Y|
| RIG9AWFOGRDVO|          2|            1|          1|   N|                Y|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|                Y|
| RB15NBVY5ELVW|          5|            2|          2|   N|                Y|
| R56358YM1ZJ7I|          5|            0|          0|   N|                N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|     

In [93]:
vine_df.agg(F.count("review_id"),F.countDistinct("review_id"),F.sum("star_rating"),F.sum("helpful_votes"),F.sum("total_votes")).collect()

[Row(count(review_id)=2402431, count(review_id)=2402431, sum(star_rating)=10359832, sum(helpful_votes)=3712495, sum(total_votes)=4733255)]

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [95]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.crjqhmcwnxji.us-east-2.rds.amazonaws.com:5432/<database name> "
config = {"user":"postgres", 
          "password": "<passowrd>", 
          "driver":"org.postgresql.Driver"}

In [96]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [97]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [100]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [101]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)